In [6]:
!pip uninstall -y opencv-python-headless opencv-python opencv-contrib-python mediapipe numpy jax jaxlib jax-cuda12-plugin protobuf -q

# Stable versions that play nicely together
!pip install --no-cache-dir \
  numpy==1.26.4 \
  "protobuf>=4.25.3,<5" \
  mediapipe==0.10.21 \
  opencv-contrib-python==4.11.0.86 -q



In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, module="jaxlib.plugin_support")

import sys, platform, time
import cv2, numpy as np, mediapipe as mp

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return 360 - angle if angle > 180.0 else angle

def open_camera_for_jupyter():
    osname = platform.system().lower()
    # try OS-preferred backends first
    if osname.startswith("win"):
        backends = [cv2.CAP_DSHOW, cv2.CAP_MSMF, None]
    elif osname == "darwin":
        backends = [cv2.CAP_AVFOUNDATION, None]
    else:
        backends = [cv2.CAP_V4L2, None]

    for b in backends:
        for idx in (0,1,2):
            cap = cv2.VideoCapture(idx) if b is None else cv2.VideoCapture(idx, b)
            if cap.isOpened():
                cap.set(cv2.CAP_PROP_FRAME_WIDTH,  640)
                cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
                cap.set(cv2.CAP_PROP_FPS, 30)
                return cap
            cap.release()
    return None

# thresholds (tune if needed)
TOP_LOCK   = 165   # knee angle considered standing
BOTTOM_TH  = 95    # depth threshold (lower -> deeper)
TORSO_WARN = 60    # forward lean cue
VIS_THR    = 0.7

cap = open_camera_for_jupyter()
if cap is None:
    raise RuntimeError(
        "Could not open webcam. Close Zoom/Meet/OBS, allow camera permissions, "
        "or try a different USB port. If a CV2 window still won't show in Jupyter, "
        "use the inline fallback in the next cell."
    )

# video writer (optional: saves your session)
ret, probe = cap.read()
if not ret:
    cap.release()
    raise RuntimeError("Webcam opened but no frames received.")
h, w = probe.shape[:2]
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("squat_session_smart.mp4", fourcc, 20.0, (w, h))

counter, state, feedback = 0, "get_ready", ""

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            avg_knee_angle, avg_torso_angle = 0, 0

            try:
                lm = results.pose_landmarks.landmark

                LHIP, LKNE, LANK, LSHO = lm[23], lm[25], lm[27], lm[11]
                RHIP, RKNE, RANK, RSHO = lm[24], lm[26], lm[28], lm[12]

                l_hip, l_knee, l_ankle = [LHIP.x, LHIP.y], [LKNE.x, LKNE.y], [LANK.x, LANK.y]
                r_hip, r_knee, r_ankle = [RHIP.x, RHIP.y], [RKNE.x, RKNE.y], [RANK.x, RANK.y]
                l_sho, r_sho = [LSHO.x, LSHO.y], [RSHO.x, RSHO.y]

                l_knee_angle = calculate_angle(l_hip, l_knee, l_ankle)
                r_knee_angle = calculate_angle(r_hip, r_knee, r_ankle)
                avg_knee_angle = (l_knee_angle + r_knee_angle) / 2

                l_torso_angle = calculate_angle(l_sho, l_hip, l_knee)
                r_torso_angle = calculate_angle(r_sho, r_hip, r_knee)
                avg_torso_angle = (l_torso_angle + r_torso_angle) / 2

                hip_below_knee = (l_hip[1] > l_knee[1]) and (r_hip[1] > r_knee[1])

                is_visible = all(pt.visibility > VIS_THR for pt in [LHIP, LKNE, RHIP, RKNE])

                # state machine
                if state == "get_ready":
                    if is_visible and avg_knee_angle > TOP_LOCK:
                        state, feedback = "ready", "READY"
                    else:
                        feedback = "STAND STRAIGHT TO START"

                elif state == "ready":
                    feedback = "GOOD FORM"
                    if avg_knee_angle < BOTTOM_TH or hip_below_knee:
                        state = "down"

                elif state == "down":
                    feedback = "DRIVE UP"
                    if avg_knee_angle > TOP_LOCK:
                        counter += 1
                        state, feedback = "ready", "REP COUNTED!"

                if avg_torso_angle > TORSO_WARN:
                    feedback = "KEEP CHEST UP"

            except Exception:
                state, feedback = "get_ready", "NO BODY DETECTED"

            # colors
            if "COUNTED" in feedback: color = (0, 255, 255)
            elif "CHEST" in feedback: color = (0, 0, 255)
            elif "GOOD" in feedback:  color = (0, 255, 0)
            elif "READY" in feedback: color = (200, 100, 0)
            else:                     color = (128, 0, 128)

            # UI
            cv2.rectangle(image, (0, 0), (250, 73), (40, 40, 40), -1)
            cv2.putText(image, 'REPS', (15, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
            cv2.putText(image, str(counter), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2)
            cv2.putText(image, 'STATUS', (130, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
            cv2.putText(image, state.upper(), (120, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

            cv2.rectangle(image, (250, 0), (max(640, image.shape[1]), 73), color, -1)
            (tw, _), _ = cv2.getTextSize(feedback, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
            text_x = 250 + (min(image.shape[1], 640) - 250 - tw) // 2
            cv2.putText(image, feedback, (max(250, text_x), 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

            cv2.putText(image, f"KNEE:  {int(avg_knee_angle)}", (15, image.shape[0]-50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
            cv2.putText(image, f"TORSO: {int(avg_torso_angle)}", (15, image.shape[0]-20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                )

            out.write(image)
            cv2.imshow("Smart Squat Trainer", image)
            if (cv2.waitKey(10) & 0xFF) in (ord('q'), 27):  # q or ESC
                break

    finally:
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        print("Saved: squat_session_smart.mp4")
        print("Total reps:", counter)
